In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Reshape
from keras.layers import Conv1D, MaxPooling1D
from keras.optimizers import SGD, Adagrad, Adam
from keras.callbacks import EarlyStopping
import readm
import numpy as np
import sklearn.metrics
from sklearn.model_selection import train_test_split
from keras.models import model_from_yaml
from keras import backend as K
print K.backend()
import tensorflow
from keras.callbacks import * 

Using TensorFlow backend.


tensorflow


In [2]:
def keras_model():

    model = Sequential()
    
    #model.add(Reshape((n, 4), input_shape=(n*4,)))
    
    model.add(Conv1D(32, 20, padding="same", input_shape=(n, 4,)))
    print model.input_shape
    print model.output_shape
    model.add(Activation('relu'))
    print model.output_shape
    model.add(MaxPooling1D(padding="same", strides=None, pool_size=200))

    # after Max Pooling
    print model.output_shape

    model.add(Flatten())
    print model.output_shape
    model.add(Dense(512))

    # Flattened 
    print model.output_shape

    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))

    # Finally
    print model.output_shape

    model.add(Activation('sigmoid'))
    return model

In [3]:
def train_and_test(X_train, y_train, X_test, y_test, n):

    model = keras_model()
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
    hist = model.fit(X_train, y_train, validation_split=0.05, callbacks=[early_stopping], batch_size= 200, nb_epoch= 10)
    print hist.history
    probas = model.predict_on_batch(X_test_rs)
    print sklearn.metrics.average_precision_score(y_test, probas)
    return model, probas

In [7]:
n = 200
X = readm.get_matrix("/Users/divyanshisrivastava/Desktop/Scripts/deep-genomics/keras-viz/c9_all.shuf.txt")
y = readm.get_labels("/Users/divyanshisrivastava/Desktop/Scripts/deep-genomics/keras-viz/c9_all.shuf.labels")

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
y_binary_train = np.array([dp if dp <= 1 else dp-1 for dp in y_train]) 
y_binary_test = np.array([dp if dp <= 1 else dp-1 for dp in y_test])
print "Done"

Done


In [26]:
K.image_data_format()
X_train_rs = np.reshape(X_train, (-1,200,4))
X_test_rs = np.reshape(X_test, (-1,200,4))

In [28]:
print X_train
print y_train

print "Start"
model, probas = train_and_test(X_train_rs, y_binary_train, X_test, y_binary_test, n)
print "Done"

[[0 0 1 ..., 0 0 1]
 [1 0 0 ..., 1 0 0]
 [1 0 0 ..., 0 1 0]
 ..., 
 [0 0 0 ..., 0 1 0]
 [1 0 0 ..., 0 0 1]
 [0 0 1 ..., 1 0 0]]
[ 1.  0.  1. ...,  1.  2.  1.]
Start
(None, 200, 4)
(None, 200, 32)
(None, 200, 32)
(None, 1, 32)
(None, 32)
(None, 512)
(None, 1)


/Users/divyanshisrivastava/anaconda/lib/python2.7/site-packages/Keras-2.0.3-py2.7.egg/keras/models.py:834: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 21597 samples, validate on 1137 samples
Epoch 1/10
21597/21597 [==============================] - 3s - loss: 0.6943 - acc: 0.5026 - val_loss: 0.6925 - val_acc: 0.5040
Epoch 2/10
21597/21597 [==============================] - 3s - loss: 0.6913 - acc: 0.5266 - val_loss: 0.6891 - val_acc: 0.5989
Epoch 3/10
21597/21597 [==============================] - 2s - loss: 0.6851 - acc: 0.5829 - val_loss: 0.6799 - val_acc: 0.6517
Epoch 4/10
21597/21597 [==============================] - 3s - loss: 0.6660 - acc: 0.6746 - val_loss: 0.6493 - val_acc: 0.7485
Epoch 5/10
21597/21597 [==============================] - 2s - loss: 0.6023 - acc: 0.7581 - val_loss: 0.5547 - val_acc: 0.7845
Epoch 6/10
21597/21597 [==============================] - 2s - loss: 0.4817 - acc: 0.8039 - val_loss: 0.4477 - val_acc: 0.8083
Epoch 7/10
21597/21597 [==============================] - 2s - loss: 0.4001 - acc: 0.8294 - val_loss: 0.4037 - val_acc: 0.8329
Epoch 8/10
21597/21597 [==============================] - 2s -

In [13]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# compute area under the roc curve
roc_auc = sklearn.metrics.roc_auc_score(y_test, probas)
print roc_auc

# compute the roc curve
fpr, tpr, thresholds = roc_curve(y_test, probas, pos_label=1)

plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.title("ROC curve")
plt.xlabel("True Positive Rate")
plt.ylabel("False Postive Rate")
plt.savefig("ROC Curve")

0.971114162658


In [12]:
print "Trained miodel"
print model
print model.layers[-1].get_config()

Trained miodel
{'activation': 'sigmoid', 'trainable': True, 'name': 'activation_6'}


In [29]:
def complie_extractembeddingfunc(model):
    inp = model.layers[0].input
    outp = model.layers[5].output
    print inp
    print outp
    return K.function([inp, K.learning_phase()],[outp])

In [30]:
extract_embedding = complie_extractembeddingfunc(model)

Tensor("conv1d_1_input:0", shape=(?, 200, 4), dtype=float32)
Tensor("activation_2/Relu:0", shape=(?, 512), dtype=float32)


In [31]:
outp = np.array(extract_embedding([X_test_rs,0]))
print outp.shape
outvec = np.reshape(outp,(outp.shape[1],512))
print outvec.shape

np.savetxt("outvec.txt", outvec)
np.savetxt("labels.txt", y_test)

(1, 2526, 512)
(2526, 512)
